In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Uncomment these lines if you haven't downloaded the nltk data before
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load sample data
data = pd.read_csv("/content/Grammar_Correction.csv")  # Replace with actual dataset path

# Print column names to verify
print("Column names in the dataset:", data.columns)

# Check if the necessary columns are present
if 'Ungrammatical_Statement' not in data.columns or 'Error_Type' not in data.columns:
    raise KeyError("The dataset must contain 'Ungrammatical_Statement' and 'Error_Type' columns")

# Preprocessing function
def preprocess_text(Ungrammatical_Statement):
    # Lowercase conversion
    # Ungrammatical_Statement = Ungrammatical_Statement.lower()
    # Remove special characters
    Ungrammatical_Statement = re.sub(r'\W', ' ', Ungrammatical_Statement)
    # Tokenization
    words = word_tokenize(Ungrammatical_Statement)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Apply preprocessing
data['cleaned_text'] = data['Ungrammatical_Statement'].apply(preprocess_text)

# Label encoding (assuming binary classification for errors)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['Error_Type'])

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Apply K-means clustering
num_clusters = 10  # Set the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(tfidf_matrix)

# Add cluster labels as features
X = np.hstack((tfidf_matrix.toarray(), data['cluster'].values.reshape(-1, 1)))
y = data['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

# Get the best model
best_rf_model = random_search.best_estimator_

# Evaluate the best model on the test data
y_pred_rf = best_rf_model.predict(X_test)
print("Random Forest Model evaluation:")
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")

# Load the pre-trained T5 model and tokenizer fine-tuned for grammar correction
model_name = 'vennify/t5-base-grammar-correction'  # Known model for grammar correction
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to correct grammatical errors using the T5 model
def correct_grammar(text, model, tokenizer):
    input_text = "gec: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

# Function to detect errors and correct the sentence
def detect_and_correct_errors(text, rf_model, vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder):
    # Preprocess the text for error detection
    cleaned_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([cleaned_text])
    text_cluster = kmeans.predict(text_tfidf)[0]
    text_features = np.hstack((text_tfidf.toarray(), np.array([[text_cluster]])))
    # Detect the error type
    predicted_label = rf_model.predict(text_features)
    # Check if the sentence has no errors based on some threshold
    if predicted_label == 0:  # Assuming 0 is the label for "no error"
        return "No error in the sentence", text
    error_type = label_encoder.inverse_transform(predicted_label)[0]
    # Correct the sentence using the T5 model
    corrected_sentence = correct_grammar(text, t5_model, t5_tokenizer)
    return error_type, corrected_sentence

# Take user input for testing the model
while True:
    user_input = input("Enter a sentence for error detection and correction (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    detected_error_type, corrected_sentence = detect_and_correct_errors(user_input, best_rf_model, tfidf_vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder)
    print(f"Detected Error Type: {detected_error_type}")
    print(f"Corrected Sentence: {corrected_sentence}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Column names in the dataset: Index(['Serial Number', 'Error_Type', 'Ungrammatical_Statement',
       'Standard_English'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


...

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

Enter a sentence for error detection and correction (or type 'exit' to quit): I goes to the store everyday.
Detected Error Type: Verb Tense Errors
Corrected Sentence: I go to the store everyday.
Enter a sentence for error detection and correction (or type 'exit' to quit): I wanting a coffee.
Detected Error Type: Capitalization Errors
Corrected Sentence: I want a coffee.
Enter a sentence for error detection and correction (or type 'exit' to quit): The family is visiiting a famous landmarks on a vacation.
Detected Error Type: Article Usage
Corrected Sentence: The family is visiting famous landmarks on a vacation.
Enter a sentence for error detection and correction (or type 'exit' to quit): The workers are building the roads.
Detected Error Type: Preposition Usage
Corrected Sentence: The workers are building the roads.
Enter a sentence for error detection and correction (or type 'exit' to quit): exit
